In [2]:
from tinygrad import Tensor, dtypes

In [14]:
a = Tensor.eye(3)
print(a.numpy())

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [91]:
dim = 3
b = Tensor.full((dim, 1), 1)
# args = ((0,0),(0,dim))
# print(tuple([(p[0], s+p[1]) for s,p in zip(b.shape, args)]))
# print(tuple(x if x is not None else (1,1) for x in args))
print(b)
print(f"{b.numpy()} full")
# arg = ((top, bottom),(left, right))
b = b.pad(((0,0),(0,dim)))
print(b)
print(f"{b.numpy()} pad")
b = b.reshape(dim*(dim+1))
print(b)
print(f"{b.numpy()} reshape")
b = b.shrink(((0, dim*dim),))
print(b)
print(f"{b.numpy()} shrink")
b = b.reshape(dim, dim)
print(b)
print(f"{b.numpy()} reshape")

<Tensor <LB METAL (3, 1) contig:False ShapeTracker(views=(View(shape=(3, 1), strides=(0, 0), offset=0, mask=None, contiguous=False),))> on METAL with grad None>
[[1]
 [1]
 [1]] full
<Tensor <LB METAL (3, 4) contig:False ShapeTracker(views=(View(shape=(3, 4), strides=(0, 0), offset=0, mask=((0, 3), (0, 1)), contiguous=False),))> on METAL with grad None>
[[1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]] pad
<Tensor <LB METAL (12,) contig:False ShapeTracker(views=(View(shape=(3, 4), strides=(0, 0), offset=0, mask=((0, 3), (0, 1)), contiguous=False), View(shape=(12,), strides=(1,), offset=0, mask=None, contiguous=True)))> on METAL with grad None>
[1 0 0 0 1 0 0 0 1 0 0 0] reshape
<Tensor <LB METAL (9,) contig:False ShapeTracker(views=(View(shape=(3, 4), strides=(0, 0), offset=0, mask=((0, 3), (0, 1)), contiguous=False), View(shape=(9,), strides=(1,), offset=0, mask=None, contiguous=True)))> on METAL with grad None>
[1 0 0 0 1 0 0 0 1] shrink
<Tensor <LB METAL (3, 3) contig:False ShapeTracker(views=(View(s

In [104]:
a = Tensor([1,1,1,1,1])
b = Tensor([1,1]).pad(((0,3),))
print(a)
print(b)
print(a.numpy(), b.numpy())
c = a*b
print(c)
print(c.numpy())


<Tensor <LB METAL (5,) contig:True (<LoadOps.COPY: 3>, None)> on METAL with grad None>
<Tensor <LB METAL (5,) contig:False ShapeTracker(views=(View(shape=(5,), strides=(1,), offset=0, mask=((0, 2),), contiguous=False),))> on METAL with grad None>
[1 1 1 1 1] [1 1 0 0 0]
<Tensor <LB METAL (5,) contig:True (<BinaryOps.MUL: 3>, None)> on METAL with grad None>
[1 1 0 0 0]


In [109]:
a = Tensor([1, 2, 3, 4, 5])
print(a.mean().numpy(), a.var().numpy(), a.std().numpy())

3.0 2.5 1.5811388


In [124]:
# mean
m = a.sum()/a.numel()
assert a.mean().numpy() == m.numpy()

# var
correction = 1
ss = (a- m).square().sum()
v = ss.div(max(0, a.numel() - correction))
assert a.var().numpy() == v.numpy()

# std
std = v.sqrt()
assert a.std().numpy() == std.numpy()

In [131]:
print(a.shape, *a.shape)
u = Tensor.rand(*a.shape) < 0.5
b = a * u
print(u.numpy(), b.numpy())

(5,) 5
[False  True  True  True False] [0 2 3 4 0]


In [136]:
from tinygrad.helpers import make_pair
stride = 1
b = Tensor.ones((1, 3, 16, 16))
print(len(b.shape))
make_pair(stride, len(b.shape))

4


(1, 1, 1, 1)

In [148]:
a = Tensor([1, 2, 3, 4, 5])
print(a.numpy(), a.shape, len(a.shape))
print(a.slice([(4,11)]).numpy())
b = Tensor.ones((2, 5))
b = b.pad([(2,2), (1, 1)])
print("c")
c = a.pad([(1, 1)])
print(c.numpy())
# print(c.shrink([(1, 6)]).numpy())
print(c.pad([(0, 1)]).numpy())

# print(b.numpy())
# print(b.shrink([(2, 4), (1, 6)]).numpy())
# print(b.slice([(2, 4), (1, 15)]).numpy())


[1 2 3 4 5] (5,) 1
[5 0 0 0 0 0 0]
c
[0 1 2 3 4 5 0]
[0 1 2 3 4 5 0 0]


In [151]:
from typing import Tuple, Union
from tinygrad.helpers import argfix, make_pair, flatten, prod, all_int, round_up, merge_dicts, fully_flatten, flat_mv, argsort
import math
sint = int

def _pool(self, k_:Tuple[sint, ...], stride:Union[Tuple[int, ...], int]=1, dilation:Union[Tuple[int, ...], int]=1) -> Tensor:
  print(self.shape, k, stride, dilation)
  assert len(self.shape) >= len(k_), f"can't pool {self.shape} with {k_}"
  assert all_int(self.shape) and all_int(k_), f"does not support symbolic {self.shape=}, {k_=}"
  s_, d_ = make_pair(stride, len(k_)), make_pair(dilation, len(k_))
  print("s ", s_)
  assert len(k_) == len(s_) == len(d_), f"stride/dilation mismatch kernel:{k_} stride:{s_} dilation:{d_}"
  noop_, i_ = [None] * len(self.shape[:-len(k_)]), self.shape[-len(k_):]
  print("noop ", noop_)
  print("i ", i_)
  if any(k > s for k,s in zip(k_, s_)) or any(d != 1 for d in d_):
    o_ = [(i - d * (k-1) - 1)//s + 1 for i,d,k,s in zip(i_, d_, k_, s_)]
    print("o ", o_)
    # repeats such that we don't need padding
    xup = self.repeat([1]*len(noop_) + [math.ceil(k*(i+d) / i) for k,i,d in zip(k_, i_, d_)])
    print("repeat", xup.shape, [math.ceil(k*(i+d) / i) for k,i,d in zip(k_, i_, d_)])
    # slice by dilation
    print("handle dilation")
    xup = xup.shrink(noop_ + [(0,k*(i+d)) for k,i,d in zip(k_, i_, d_)])
    print("slice", xup.shape)
    xup = xup.reshape(noop_ + flatten((k,i+d) for k,i,d in zip(k_, i_, d_)))
    print("reshape", xup.shape)
    # handle stride
    print("handle stride")
    xup = xup.shrink(noop_ + flatten(((0,k), (0,o*s)) for k,o,s in zip(k_, o_, s_)))
    print("slice", xup.shape)
    xup = xup.reshape(noop_ + flatten((k,o,s) for k,o,s in zip(k_, o_, s_)))
    print("reshape", xup.shape)
    xup = xup.shrink(noop_ + flatten(((0,k), (0,o), (0,1)) for k,o in zip(k_, o_)))
    print("slice", xup.shape)
    xup = xup.reshape(noop_ + flatten((k,o) for k,o in zip(k_, o_)))
    print("reshape", xup.shape)
    # permute to move reduce to the end
    print("permute")
    return xup.permute(*range(len(noop_)), *[len(noop_)+i*2+1 for i in range(len(i_))], *[len(noop_)+i*2 for i in range(len(i_))])
  # TODO: once the shapetracker can optimize well, remove this alternative implementation. or not if the CPU implementation doesn't use ShapeTracker
  o_ = [(i+(s-k))//s for i,s,k in zip(i_, s_, k_)]
  print("outer o", o_)
  # TODO: remove slice and use pad and shrink
  # xup = self.slice(noop_ + [(0,o*s) for o,s in zip(o_, s_)])
  print(self.shape, [(0, max(0,o*s-i)) for i,o,s in zip(i_, o_, s_)])
  xup = self.pad(noop_ + [(0, max(0,o*s-i)) for i,o,s in zip(i_, o_, s_)])
  print("pad", xup.shape)
  xup = xup.shrink(noop_ + [(0,o*s) for o,s in zip(o_, s_)])  
  # xup = xup.shrink(noop_ + arg)
  print("outer slice", xup.shape)
  # idxs = noop_ + [(0,o*s) for o,s in zip(o_, s_)]
  # print(idxs)
  # xup = self.pad(tuple((max(0,-a+b[0]), max(0,b[1]-a)) for a,b in zip(i_,idxs))).shrink(idxs)
  # print(noop_, xup.numpy(), [(0,o*s) for o,s in zip(o_, s_)])
  xup = xup.reshape(noop_ + flatten(((o,s) for o,s in zip(o_, s_))))
  print("outer reshape", xup.shape)
  xup = xup.shrink(noop_ + flatten(((0,o), (0,k)) for o,k in zip(o_, k_)))
  print("outer slice", xup.shape)
  return xup.permute(*range(len(noop_)), *[len(noop_)+i*2 for i in range(len(i_))], *[len(noop_)+i*2+1 for i in range(len(i_))])

a = Tensor.ones((32,2,110,28))
k = (3, 3)
stride = (3,2)
dilation = 1
c = _pool(a, k, stride, dilation)
print(c.shape, "(bs, groups*cin, oy, ox, H, W)")
# print(c.numpy())

(32, 2, 110, 28) (3, 3) (3, 2) 1
s  (3, 2)
noop  [None, None]
i  (110, 28)
o  [36, 13]
repeat (32, 2, 440, 112) [4, 4]
handle dilation
slice (32, 2, 333, 87)
reshape (32, 2, 3, 111, 3, 29)
handle stride
slice (32, 2, 3, 108, 3, 26)
reshape (32, 2, 3, 36, 3, 3, 13, 2)
slice (32, 2, 3, 36, 1, 3, 13, 1)
reshape (32, 2, 3, 36, 3, 13)
permute
(32, 2, 36, 13, 3, 3) (bs, groups*cin, oy, ox, H, W)


In [ ]:
def _pool(self, k_:Tuple[sint, ...], stride:Union[Tuple[int, ...], int]=1, dilation:Union[Tuple[int, ...], int]=1) -> Tensor:
    assert len(self.shape) >= len(k_), f"can't pool {self.shape} with {k_}"
    assert all_int(self.shape) and all_int(k_), f"does not support symbolic {self.shape=}, {k_=}"
    s_, d_ = make_pair(stride, len(k_)), make_pair(dilation, len(k_))
    assert len(k_) == len(s_) == len(d_), f"stride/dilation mismatch kernel:{k_} stride:{s_} dilation:{d_}"
    noop_, i_ = [None] * len(self.shape[:-len(k_)]), self.shape[-len(k_):]
    if any(k > s for k,s in zip(k_, s_)) or any(d != 1 for d in d_):
      o_ = [(i - d * (k-1) - 1)//s + 1 for i,d,k,s in zip(i_, d_, k_, s_)]
      # repeats such that we don't need padding
      xup = self.repeat([1]*len(noop_) + [math.ceil(k*(i+d) / i) for k,i,d in zip(k_, i_, d_)])
      # slice by dilation
      xup = xup.shrink(noop_ + [(0,k*(i+d)) for k,i,d in zip(k_, i_, d_)]).reshape(noop_ + flatten((k,i+d) for k,i,d in zip(k_, i_, d_)))
      # handle stride
      xup = xup.shrink(noop_ + flatten(((0,k), (0,o*s)) for k,o,s in zip(k_, o_, s_))).reshape(noop_ + flatten((k,o,s) for k,o,s in zip(k_, o_, s_)))
      xup = xup.shrink(noop_ + flatten(((0,k), (0,o), (0,1)) for k,o in zip(k_, o_))).reshape(noop_ + flatten((k,o) for k,o in zip(k_, o_)))
      # permute to move reduce to the end
      return xup.permute(*range(len(noop_)), *[len(noop_)+i*2+1 for i in range(len(i_))], *[len(noop_)+i*2 for i in range(len(i_))])
    # TODO: once the shapetracker can optimize well, remove this alternative implementation. or not if the CPU implementation doesn't use ShapeTracker
    o_ = [(i+(s-k))//s for i,s,k in zip(i_, s_, k_)]
    xup = self.pad(noop_ + [(0, max(0,o*s-i)) for i,o,s in zip(i_, o_, s_)]).shrink(noop_ + [(0,o*s) for o,s in zip(o_, s_)])
    xup = xup.reshape(noop_ + flatten(((o,s) for o,s in zip(o_, s_))))
    xup = xup.shrink(noop_ + flatten(((0,o), (0,k)) for o,k in zip(o_, k_)))
    return xup.permute(*range(len(noop_)), *[len(noop_)+i*2 for i in range(len(i_))], *[len(noop_)+i*2+1 for i in range(len(i_))])


def test_maxpool2d_bigger_stride_dilation(self):
    for stride, dilation in zip([(2,3), (3,2), 2, 3, 4], [(3,2), (2,3), 2, 3, 6]):
      with self.subTest(stride=stride):
        helper_test_op([(32,2,110,28)],
          lambda x: torch.nn.functional.max_pool2d(x, kernel_size=(2,2), stride=stride, dilation=dilation),
          lambda x: Tensor.max_pool2d(x, kernel_size=(2,2), stride=stride, dilation=dilation))

In [61]:
make_pair(k, a.shape)
assert a.avg_pool2d((3,3),1,0).shape == c.mean(axis=(-2,-1)).shape
assert a.max_pool2d((3,3),1,0).shape == c.max(axis=(-2,-1)).shape

In [29]:
a = Tensor.ones((1, 3, 6, 6))
a = a.repeat((1, 1, 3, 3))
print(a.shape)
a.reshape(1, 3, 3, 6, 3, 6)

(1, 3, 18, 18)


<Tensor <LB METAL (1, 3, 3, 6, 3, 6) contig:False ShapeTracker(views=(View(shape=(1, 3, 3, 6, 3, 6), strides=(0, 0, 0, 0, 0, 0), offset=0, mask=None, contiguous=False),))> on METAL with grad None>

In [34]:
a = ((1,1), (1,1))
b = ((0,0), (0,0))
flatten((x,y) for x,y in zip(a, b))

[(1, 1), (0, 0), (1, 1), (0, 0)]

In [69]:
i,d,k,s = 6,0,3,2
res = (i - d * (k-1) - 1)//s + 1
print(res)

3


In [21]:
a = Tensor([[1], [2,3 ]])
a.expand((a.shape[0], 4)).numpy()

array([[1, 1, 1, 1],
       [2, 2, 2, 2]], dtype=int32)